# Instagram Post Scraper

In [156]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs

In [157]:
browser  = webdriver.Firefox()
browser.get('https://www.instagram.com/')

In [158]:
WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='username']")))
username = browser.find_element(By.CSS_SELECTOR, "input[name='username']")
password = browser.find_element(By.CSS_SELECTOR, "input[name='password']")
username.clear()
password.clear()
username.send_keys("ithinktheytoldthewarden")
password.send_keys("lndlndlnd")

In [159]:
time.sleep(5)
login_button = browser.find_element(By.CSS_SELECTOR, "button[type='submit']")
login_button.click()

In [160]:
time.sleep(5)

In [161]:
username='donttellthewarden'
browser.get('https://www.instagram.com/'+username+'/?hl=en')
links = set()

In [162]:
n_scrolls = 20
for _ in range(n_scrolls):
    source = browser.page_source
    data=bs(source, 'html.parser')
    posts = data.find_all('a', class_='_a6hd')
    time.sleep(15)
    
    for post in posts:
        href = post.get('href')
        if href.startswith('/p/') or href.startswith('/reels/'):
            link = "https://instagram.com" + href
            links.add(link)
            
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [163]:
print(links)

{'https://instagram.com/p/CVxmEKAPLKo/?hl=en', 'https://instagram.com/p/C7mE2Yxv5JS/?hl=en', 'https://instagram.com/p/CQqvOgCMUye/?hl=en', 'https://instagram.com/p/CVsP373vkMm/?hl=en', 'https://instagram.com/p/CByGNmpjFSY/?hl=en', 'https://instagram.com/p/Ct9Zmodsk-1/?hl=en', 'https://instagram.com/p/C2KM8Vov7-N/?hl=en', 'https://instagram.com/p/CJEBX9PDnZT/?hl=en', 'https://instagram.com/p/CO7gMuAjAPz/?hl=en', 'https://instagram.com/p/CJWGONaD8St/?hl=en', 'https://instagram.com/p/CB12LZBDU-r/?hl=en', 'https://instagram.com/p/C4aqJIdvE1N/?hl=en', 'https://instagram.com/p/CsqtpaWvBGF/?hl=en', 'https://instagram.com/p/CbkHMknPQO6/?hl=en', 'https://instagram.com/p/CU5JflxPqxO/?hl=en', 'https://instagram.com/p/CGzxQi6D8-X/?hl=en', 'https://instagram.com/p/Cjx4RuLPdkQ/?hl=en', 'https://instagram.com/p/CXtcYmrvGse/?hl=en', 'https://instagram.com/p/CPx8GCsDTJy/?hl=en', 'https://instagram.com/p/Ct__UPEPFEs/?hl=en', 'https://instagram.com/p/CfjLiNmBDzc/?hl=en', 'https://instagram.com/p/CGVBF5Ti

In [164]:
print(len(links))

162


In [165]:
data_list = []

for i, link in enumerate(links):
    # if i >= 20:
        # break

    browser.get(link)
    time.sleep(5)

    source = browser.page_source
    data=bs(source, 'html.parser')
    
    time_elements = data.find_all('time', class_='x1p4m5qa')
    
    for time_element in time_elements:
        datetime_attr = time_element.get('datetime')
        title_attr = time_element.get('title')
        
        dt_object = datetime.fromisoformat(datetime_attr.rstrip('Z'))
        date_of_posting = dt_object.date()
        time_of_posting = dt_object.time()
        
        time_in_IST = (datetime.combine(date_of_posting, time_of_posting) + timedelta(hours=4, minutes=30)).time()
            
        likes_element = data.find('span', class_='html-span xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1hl2dhg x16tdsg8 x1vvkbs')
        likes = likes_element.text if likes_element else '0'  
        
        data_list.append({
                'title': title_attr,
                'date_of_posting': date_of_posting,
                'time_of_posting': time_of_posting,
                'time_in_IST': time_in_IST,
                'likes': likes
            })

In [166]:
print(data_list)

[{'title': 'Nov 2, 2021', 'date_of_posting': datetime.date(2021, 11, 2), 'time_of_posting': datetime.time(13, 20, 9), 'time_in_IST': datetime.time(17, 50, 9), 'likes': '41'}, {'title': 'May 30, 2024', 'date_of_posting': datetime.date(2024, 5, 30), 'time_of_posting': datetime.time(13, 51, 15), 'time_in_IST': datetime.time(18, 21, 15), 'likes': '95'}, {'title': 'Jun 28, 2021', 'date_of_posting': datetime.date(2021, 6, 28), 'time_of_posting': datetime.time(14, 48, 33), 'time_in_IST': datetime.time(19, 18, 33), 'likes': '168'}, {'title': 'Oct 31, 2021', 'date_of_posting': datetime.date(2021, 10, 31), 'time_of_posting': datetime.time(11, 30, 2), 'time_in_IST': datetime.time(16, 0, 2), 'likes': '160'}, {'title': 'Jun 23, 2020', 'date_of_posting': datetime.date(2020, 6, 23), 'time_of_posting': datetime.time(15, 33, 11), 'time_in_IST': datetime.time(20, 3, 11), 'likes': '105'}, {'title': 'Jun 26, 2023', 'date_of_posting': datetime.date(2023, 6, 26), 'time_of_posting': datetime.time(15, 55, 36)

# Data Analysis

In [167]:
import pandas as pd

In [168]:
df = pd.DataFrame(data_list)
df.set_index('title', inplace=True)
df.sort_values(by='date_of_posting', ascending=False, inplace=True)

In [169]:
df

,date_of_posting,time_of_posting,time_in_IST,likes
title,,,,
"Jun 16, 2024",2024-06-16,08:00:07,12:30:07,47
"Jun 11, 2024",2024-06-11,15:02:34,19:32:34,80
"Jun 1, 2024",2024-06-01,15:37:01,20:07:01,136
"May 31, 2024",2024-05-31,15:18:35,19:48:35,113
"May 30, 2024",2024-05-30,13:51:15,18:21:15,95
...,...,...,...,...
"Jun 27, 2020",2020-06-27,16:04:20,20:34:20,87
"Jun 26, 2020",2020-06-26,15:54:03,20:24:03,128
"Jun 25, 2020",2020-06-25,02:30:02,07:00:02,145


In [170]:
df.to_csv('postData.csv')